# XRR Lineup - RSoXS Beamline Alignment

This notebook implements the automated XRR Lineup alignment procedure for the RSoXS beamline.

## 3-Motor Alignment Procedure:

1. **CCD Theta** - Find direct beam maximum (single iteration)

   - Scans CCD Theta to find angle that maximizes photodiode signal
   - Peak function profile

2. **Sample Z** - Find 50% intensity point (iterative)
   - Scans Sample Z to find height where beam intensity is cut in half
   - Sigmoid/error function profile
3. **Sample Theta** - Find maximum intensity (iterative)
   - Scans Sample Theta to find angle that maximizes scattered intensity
   - Peak function profile

**Steps 2 and 3 are repeated iteratively with progressively finer step sizes** (4 iterations total) to achieve precise alignment of the sample's axis of rotation with the incident beam.


In [3]:
# Import necessary modules
import asyncio
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from api_dev import RsoxsServer as Server
from api_dev import (
    create_line_scan,
    find_peak_position,
)

# Connect to the beamline
server = await Server.create()
print("✓ Connected to RSoXS beamline")


@dataclass
class AlignmentConfig:
    """Configuration parameters for the alignment algorithm."""

    # Initial positions
    initial_ccd_theta: float = 0.0
    initial_sample_theta: float = 0.0
    initial_sample_z: float = 0.0
    initial_ccd_x: float = 6.0  # Ensures we are on the photodiode

    # CCD theta scan parameters
    ccd_theta_range: Tuple[float, float] = (-2.0, 2.0)
    ccd_theta_step: float = 0.1

    # Sample theta scan parameters
    sample_theta_range: Tuple[float, float] = (0.0, 2.0)
    sample_theta_step: float = 0.05
    ccd_theta_offset: float = 2.0  # CCD position for sample theta alignment

    # Alignment parameters
    target_intensity_fraction: float = 0.5  # Cut beam intensity in half
    sample_z_step_initial: float = 0.1  # Initial step size for sample height
    sample_x_step_initial: float = 0.05  # Initial step size for sample X

    # Convergence criteria
    intensity_tolerance: float = 0.02  # 2% tolerance for intensity matching
    position_tolerance: float = 0.001  # 1 µm tolerance for position
    max_iterations: int = 10

    # Safety limits
    sample_z_max: float = 20.0  # Maximum sample height
    sample_x_max: float = 50.0  # Maximum sample X position

    # Signal channels
    photodiode_channel: str = "Photodiode"
    ai_channels: List[str] = None

    # Shutter control
    shutter_channel: str = "Shutter Output"

    def __post_init__(self):
        if self.ai_channels is None:
            self.ai_channels = [self.photodiode_channel]


@dataclass
class AlignmentResult:
    """Results from alignment procedure."""

    ccd_theta_center: float
    sample_z_aligned: float
    sample_theta_offset: float
    final_sample_x: float
    final_sample_theta: float

    # Convergence info
    iterations_completed: int
    converged: bool
    final_intensity_error: float

    # Intermediate results
    photodiode_scan_data: Optional[pd.DataFrame] = None
    height_alignment_data: Optional[pd.DataFrame] = None
    theta_alignment_data: Optional[pd.DataFrame] = None
    refinement_history: Optional[List[Dict]] = None


class BeamlineAligner:
    """
    XRR Lineup alignment algorithm for RSoXS beamline.

    Implements the 3-motor alignment procedure:
    1. CCD Theta - Find direct beam maximum (single iteration)
    2. Sample Z - Find half-intensity point (sigmoid/error function)
    3. Sample Theta - Find maximum intensity (peak function)

    Steps 2 and 3 are iteratively refined with decreasing step sizes.
    """

    def __init__(self, server: Server, config: AlignmentConfig = None):
        self.server = server
        self.config = config or AlignmentConfig()
        self.refinement_history = []

    async def full_alignment(
        self, exposure_time: float = 0.2, plot_results: bool = True
    ) -> AlignmentResult:
        """
        Execute complete XRR lineup alignment procedure.

        Parameters
        ----------
        exposure_time : float
            Exposure time for measurements
        plot_results : bool
            Whether to plot intermediate results

        Returns
        -------
        AlignmentResult
            Complete alignment results and diagnostics
        """
        print("🎯 Starting XRR Lineup Alignment...")
        print("=" * 50)

        # Initialize positions and open shutter
        await self._initialize_positions()
        # Step 1: CCD Theta alignment (single iteration)
        print("📍 Step 1: CCD Theta alignment - finding direct beam maximum...")
        ccd_theta_center, ccd_scan_data = await self._align_ccd_theta(exposure_time)
        print(f"   ✓ CCD Theta aligned: {ccd_theta_center:.3f}°")

        # Step 2: Iterative Sample Z and Sample Theta alignment
        print("🔄 Step 2: Iterative Sample Z and Sample Theta alignment...")

        # Alignment iterations with decreasing step sizes
        iterations = [
            {
                "z_range": 1.0,
                "z_step": 0.1,
                "th_range": 2.0,
                "th_step": 0.2,
                "name": "1",
            },
            {
                "z_range": 1.0,
                "z_step": 0.1,
                "th_range": 1.5,
                "th_step": 0.05,
                "name": "2",
            },
            {
                "z_range": 1.0,
                "z_step": 0.05,
                "th_range": 1.2,
                "th_step": 0.025,
                "name": "3",
            },
            {
                "z_range": 0.8,
                "z_step": 0.02,
                "th_range": 1.0,
                "th_step": 0.02,
                "name": "4",
            },
        ]

        z_scan_data = []
        theta_scan_data = []

        for iter_params in iterations:
            print(f"\n   Iteration {iter_params['name']}:")

            # Sample Z alignment (50% intensity point)
            print(
                f"   📏 Sample Z scan (range: ±{iter_params['z_range']}, step: {iter_params['z_step']})"
            )
            z_result, z_data = await self._align_sample_z(
                range_mm=iter_params["z_range"],
                step_size=iter_params["z_step"],
                exposure_time=exposure_time,
            )
            z_scan_data.append(z_data)
            print(f"      → Sample Z aligned: {z_result:.4f} mm")

            # Move CCD to 2θ position for Sample Theta scan
            await self.server.motor.set("CCD Theta", 2.0)  # 2θ position
            await self.server.motor.set("Sample Theta", 1.0)  # Starting position
            await asyncio.sleep(1.0)

            # Sample Theta alignment (maximum intensity)
            print(
                f"   🎯 Sample Theta scan (range: ±{iter_params['th_range']}, step: {iter_params['th_step']})"
            )
            theta_result, theta_data = await self._align_sample_theta(
                range_deg=iter_params["th_range"],
                step_size=iter_params["th_step"],
                exposure_time=exposure_time,
            )
            theta_scan_data.append(theta_data)
            print(f"      → Sample Theta aligned: {theta_result:.4f}°")

            # Move CCD back to direct beam position
            await self.server.motor.set("CCD Theta", -2.0)
            await self.server.motor.set("Sample Theta", -1.0)
            await asyncio.sleep(1.0)

        # Calculate final intensity error
        final_intensity_error = await self._measure_intensity_error()

        # Create result object
        result = AlignmentResult(
            ccd_theta_center=ccd_theta_center,
            sample_z_aligned=z_result,
            sample_theta_offset=theta_result,  # Using as final theta position
            final_sample_x=0.0,  # Not aligned in this procedure
            final_sample_theta=theta_result,
            iterations_completed=len(iterations),
            converged=True,  # Assume converged after all iterations
            final_intensity_error=final_intensity_error,
            photodiode_scan_data=ccd_scan_data,
            height_alignment_data=z_scan_data[-1] if z_scan_data else None,
            theta_alignment_data=theta_scan_data[-1] if theta_scan_data else None,
            refinement_history=self.refinement_history.copy(),
        )

        if plot_results:
            self._plot_alignment_results(result)

        print("=" * 50)
        print("🎉 XRR Lineup Alignment Complete!")
        return result

    async def _initialize_positions(self):
        """Initialize motors to starting positions."""
        print("   Initializing motor positions...")

        # Move to starting positions
        await self.server.motor.set("Sample Theta", 0.0)
        await self.server.motor.set("CCD Theta", 0.0)
        await self.server.motor.set("Sample Z", -2.0)  # Below beam initially

        # Wait for motors to settle
        await asyncio.sleep(2.0)
        print("   ✓ Motors initialized")

    async def _align_ccd_theta(
        self, exposure_time: float
    ) -> Tuple[float, pd.DataFrame]:
        """
        Step 1: CCD Theta alignment - find direct beam maximum.

        Single scan to find CCD angle that maximizes photodiode signal.
        """
        # Create CCD theta scan (relative scan around 0°)
        scan_df = create_line_scan(
            motor="CCD Theta",
            start=-2.0,  # Relative to current position
            stop=2.0,
            num_points=int(4.0 / 0.1) + 1,  # 0.1° steps
            exposure_time=exposure_time,
        )

        # Execute scan - Fix: remove 'plan=' parameter
        results = await self.server.scan_from_dataframe(
            scan_df, ai_channels=self.config.ai_channels, progress=True
        )

        # Find peak position (maximum intensity)
        peak_theta, peak_intensity = find_peak_position(
            results,
            motor_col="CCD Theta_position",
            signal_col=f"{self.config.photodiode_channel}_mean",
        )

        # Move to peak position
        await self.server.motor.set("CCD Theta", peak_theta)
        await asyncio.sleep(1.0)

        return peak_theta, results

    async def _align_sample_z(
        self, range_mm: float, step_size: float, exposure_time: float
    ) -> Tuple[float, pd.DataFrame]:
        """
        Sample Z alignment - find 50% intensity point.

        Scans Sample Z to find position where intensity is 50% of maximum.
        This creates a sigmoid/error function profile.
        """
        # Get current Z position as center
        current_z = await self._get_motor_position("Sample Z")

        # Create Z scan around current position
        start_z = current_z - range_mm
        stop_z = current_z + range_mm
        num_points = int(2 * range_mm / step_size) + 1

        scan_df = create_line_scan(
            motor="Sample Z",
            start=start_z,
            stop=stop_z,
            num_points=num_points,
            exposure_time=exposure_time,
        )

        # Execute scan - Fix: remove 'df=' parameter name
        results = await self.server.scan_from_dataframe(
            scan_df, ai_channels=self.config.ai_channels, progress=True
        )

        # Find 50% intensity point
        intensities = results[f"{self.config.photodiode_channel}_mean"].values
        positions = results["Sample Z_position"].values

        # Find min and max intensities
        min_intensity = np.min(intensities)
        max_intensity = np.max(intensities)
        target_intensity = min_intensity + 0.5 * (max_intensity - min_intensity)

        # Find position closest to 50% intensity
        idx_50 = np.argmin(np.abs(intensities - target_intensity))
        z_50_position = positions[idx_50]

        # Move to 50% position
        await self.server.motor.set("Sample Z", z_50_position)
        await asyncio.sleep(1.0)

        print(f"      Min intensity: {min_intensity:.3f}")
        print(f"      Max intensity: {max_intensity:.3f}")
        print(f"      Target (50%): {target_intensity:.3f}")
        print(f"      Found at Z: {z_50_position:.4f} mm")

        return z_50_position, results

    async def _align_sample_theta(
        self, range_deg: float, step_size: float, exposure_time: float
    ) -> Tuple[float, pd.DataFrame]:
        """
        Sample Theta alignment - find maximum intensity.

        Scans Sample Theta to find angle that maximizes intensity.
        This creates a peak function profile.
        """
        # Get current theta position as center
        current_theta = await self._get_motor_position("Sample Theta")

        # Create theta scan around current position
        start_theta = current_theta - range_deg
        stop_theta = current_theta + range_deg
        num_points = int(2 * range_deg / step_size) + 1

        scan_df = create_line_scan(
            motor="Sample Theta",
            start=start_theta,
            stop=stop_theta,
            num_points=num_points,
            exposure_time=exposure_time,
        )

        # Execute scan - Fix: remove 'df=' parameter name
        results = await self.server.scan_from_dataframe(
            scan_df, ai_channels=self.config.ai_channels, progress=True
        )

        # Find peak position (maximum intensity)
        peak_theta, peak_intensity = find_peak_position(
            results,
            motor_col="Sample Theta_position",
            signal_col=f"{self.config.photodiode_channel}_mean",
        )

        # Move to peak position
        await self.server.motor.set("Sample Theta", peak_theta)
        await asyncio.sleep(1.0)

        print(f"      Peak intensity: {peak_intensity:.3f}")
        print(f"      Found at Theta: {peak_theta:.4f}°")

        return peak_theta, results

    async def _get_motor_position(self, motor_name: str) -> float:
        """Get current position of specified motor."""
        result = await self.server.motor[motor_name]
        return result["data"][0]["position"]

    async def _measure_intensity_error(self) -> float:
        """Measure final intensity error for diagnostics."""
        data = await self.server.ai.get_with_uncertainty(
            keys=self.config.ai_channels, acquisition_time=1.0
        )
        return data[self.config.photodiode_channel].std_dev

    async def _open_shutter(self):
        """Open the beam shutter."""
        print(f"   Opening shutter: {self.config.shutter_channel}")
        await self.server.dio.set(self.config.shutter_channel, True)
        await asyncio.sleep(self.config.shutter_settle_time)

    async def _close_shutter(self):
        """Close the beam shutter."""
        print(f"   Closing shutter: {self.config.shutter_channel}")
        await self.server.dio.set(self.config.shutter_channel, False)
        await asyncio.sleep(self.config.shutter_settle_time)

    async def _get_shutter_state(self) -> bool:
        """Get current shutter state. Returns True if open, False if closed."""
        result = await self.server.dio[self.config.shutter_channel]
        current_value = result["data"][0]["value"]
        return abs(current_value - self.config.shutter_open_value) < 0.1

    def _plot_alignment_results(self, result: AlignmentResult):
        """Plot comprehensive alignment results."""
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle("XRR Lineup Alignment Results", fontsize=16, fontweight="bold")

        # Plot 1: CCD Theta Scan (Direct Beam Maximum)
        if result.photodiode_scan_data is not None:
            ax = axes[0, 0]
            data = result.photodiode_scan_data
            ax.errorbar(
                data["CCD Theta_position"],
                data[f"{self.config.photodiode_channel}_mean"],
                yerr=data.get(f"{self.config.photodiode_channel}_std", 0),
                marker="o",
                capsize=3,
            )
            ax.axvline(
                result.ccd_theta_center,
                color="red",
                linestyle="--",
                label=f"Peak: {result.ccd_theta_center:.3f}°",
            )
            ax.set_xlabel("CCD Theta (°)")
            ax.set_ylabel("Photodiode Signal")
            ax.set_title("CCD Theta Alignment (Direct Beam Maximum)")
            ax.legend()
            ax.grid(True, alpha=0.3)

        # Plot 2: Sample Z Alignment (50% Intensity Point)
        if result.height_alignment_data is not None:
            ax = axes[0, 1]
            data = result.height_alignment_data
            ax.errorbar(
                data["Sample Z_position"],
                data[f"{self.config.photodiode_channel}_mean"],
                yerr=data.get(f"{self.config.photodiode_channel}_std", 0),
                marker="o",
                capsize=3,
            )

            # Mark 50% intensity point
            intensities = data[f"{self.config.photodiode_channel}_mean"].values
            min_int = np.min(intensities)
            max_int = np.max(intensities)
            target_int = min_int + 0.5 * (max_int - min_int)
            ax.axhline(
                target_int,
                color="red",
                linestyle="--",
                label=f"50% Intensity: {target_int:.3f}",
            )
            ax.axvline(
                result.sample_z_aligned,
                color="green",
                linestyle="--",
                label=f"Final Z: {result.sample_z_aligned:.3f} mm",
            )

            ax.set_xlabel("Sample Z (mm)")
            ax.set_ylabel("Photodiode Signal")
            ax.set_title("Sample Z Alignment (50% Intensity)")
            ax.legend()
            ax.grid(True, alpha=0.3)

        # Plot 3: Sample Theta Alignment (Maximum Intensity)
        if result.theta_alignment_data is not None:
            ax = axes[1, 0]
            data = result.theta_alignment_data
            ax.errorbar(
                data["Sample Theta_position"],
                data[f"{self.config.photodiode_channel}_mean"],
                yerr=data.get(f"{self.config.photodiode_channel}_std", 0),
                marker="o",
                capsize=3,
            )

            # Mark peak position
            peak_theta, peak_intensity = find_peak_position(
                data, "Sample Theta_position", f"{self.config.photodiode_channel}_mean"
            )
            ax.axvline(
                peak_theta,
                color="red",
                linestyle="--",
                label=f"Peak: {peak_theta:.3f}°",
            )

            ax.set_xlabel("Sample Theta (°)")
            ax.set_ylabel("Photodiode Signal")
            ax.set_title("Sample Theta Alignment (Maximum Intensity)")
            ax.legend()
            ax.grid(True, alpha=0.3)

        # Plot 4: Alignment Summary
        ax = axes[1, 1]
        ax.text(0.1, 0.8, "XRR Lineup Summary:", fontsize=14, fontweight="bold")
        ax.text(0.1, 0.7, f"CCD Theta: {result.ccd_theta_center:.4f}°", fontsize=12)
        ax.text(0.1, 0.6, f"Sample Z: {result.sample_z_aligned:.4f} mm", fontsize=12)
        ax.text(
            0.1, 0.5, f"Sample Theta: {result.final_sample_theta:.4f}°", fontsize=12
        )
        ax.text(0.1, 0.4, f"Iterations: {result.iterations_completed}", fontsize=12)
        ax.text(
            0.1, 0.3, f"Final Error: {result.final_intensity_error:.6f}", fontsize=12
        )
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.axis("off")

        plt.tight_layout()
        plt.show()

        # Print summary
        print("\n📊 XRR Lineup Summary:")
        print(f"   CCD Theta (Direct Beam): {result.ccd_theta_center:.4f}°")
        print(f"   Sample Z (50% Intensity): {result.sample_z_aligned:.4f} mm")
        print(f"   Sample Theta (Peak): {result.final_sample_theta:.4f}°")
        print(f"   Iterations Completed: {result.iterations_completed}")
        print(f"   Final Intensity Error: {result.final_intensity_error:.4f}")


# Create alignment configuration
config = AlignmentConfig(
    # Customize parameters as needed
    ccd_theta_range=(-2.0, 2.0),  # CCD theta scan range
    ccd_theta_step=0.1,  # CCD theta step size
    sample_theta_range=(0.0, 2.0),  # Sample theta scan range
    sample_theta_step=0.05,  # Sample theta step size
    target_intensity_fraction=0.5,  # Target intensity fraction
    max_iterations=10,  # Maximum refinement iterations
    photodiode_channel="Photodiode",  # Primary signal channel
    shutter_channel="Shutter Output",  # Shutter DIO channel
)

✓ Connected to RSoXS beamline


In [4]:
# Create the alignment engine
aligner = BeamlineAligner(server, config)

print("✓ Alignment configuration and BeamlineAligner created")
print(
    f"   CCD Theta Range: {config.ccd_theta_range[0]}° to {config.ccd_theta_range[1]}°"
)
print(
    f"   Sample Theta Range: {config.sample_theta_range[0]}° to {config.sample_theta_range[1]}°"
)
print(f"   Target Intensity: {config.target_intensity_fraction * 100}% of baseline")
print(f"   Shutter Channel: {config.shutter_channel}")
print(f"   Photodiode Channel: {config.photodiode_channel}")

# Execute the complete alignment procedure
result = await aligner.full_alignment(
    exposure_time=0.2,  # Exposure time for measurements
    plot_results=True,  # Show detailed plots
)


✓ Alignment configuration and BeamlineAligner created
   CCD Theta Range: -2.0° to 2.0°
   Sample Theta Range: 0.0° to 2.0°
   Target Intensity: 50.0% of baseline
   Shutter Channel: Shutter Output
   Photodiode Channel: Photodiode
🎯 Starting XRR Lineup Alignment...
   Initializing motor positions...
   ✓ Motors initialized
📍 Step 1: CCD Theta alignment - finding direct beam maximum...
   ✓ Motors initialized
📍 Step 1: CCD Theta alignment - finding direct beam maximum...


Scanning: 100%|██████████| 41/41 [02:50<00:00,  4.17s/pt]



   ✓ CCD Theta aligned: -0.300°
🔄 Step 2: Iterative Sample Z and Sample Theta alignment...

   Iteration 1:
   📏 Sample Z scan (range: ±1.0, step: 0.1)


Scanning: 100%|██████████| 21/21 [01:48<00:00,  5.15s/pt]



      Min intensity: 0.171
      Max intensity: 84.570
      Target (50%): 42.371
      Found at Z: -2.5000 mm
      → Sample Z aligned: -2.5000 mm
   🎯 Sample Theta scan (range: ±2.0, step: 0.2)
   🎯 Sample Theta scan (range: ±2.0, step: 0.2)


...

      Min intensity: -0.049
      Max intensity: -0.048
      Target (50%): -0.048
      Found at Z: -1.4000 mm
      → Sample Z aligned: -1.4000 mm
   🎯 Sample Theta scan (range: ±1.2, step: 0.025)
   🎯 Sample Theta scan (range: ±1.2, step: 0.025)


Scanning: 100%|██████████| 96/96 [01:47<00:00,  1.12s/pt]



      Peak intensity: -0.048
      Found at Theta: -1.8564°
      → Sample Theta aligned: -1.8564°

   Iteration 4:
   📏 Sample Z scan (range: ±0.8, step: 0.02)

   Iteration 4:
   📏 Sample Z scan (range: ±0.8, step: 0.02)


Scanning:  91%|█████████▏| 74/81 [01:47<00:10,  1.45s/pt]



CancelledError: 